**Autor:** Pedro Evaristo de Oliveira -                                                                     04/09/2025

Staffelter Hof Winery is Germany's oldest business, established in 862 under the Carolingian dynasty. It has continued to serve customers through dramatic changes in Europe, such as the Holy Roman Empire, the Ottoman Empire, and both world wars. What characteristics enable a business to stand the test of time?

To help answer this question, BusinessFinancing.co.uk researched the oldest company still in business in **almost** every country and compiled the results into several CSV files. This dataset has been cleaned.

Having useful information in different files is a common problem. While it's better to keep different types of data separate for data storage, you'll want all the data in one place for analysis. You'll use joining and data manipulation to work with this data and better understand the world's oldest businesses.

## The Data
`businesses` and `new_businesses`
|Column|Description|
|------|-----------|
|`business`|Name of the business (varchar)|
|`year_founded`|Year the business was founded (int)|
|`category_code`|Code for the business category (varchar)|
|`country_code`|ISO 3166-1 three-letter country code (char)|
---
`countries`
|Column|Description|
|------|-----------|
|`country_code`|ISO 3166-1 three-letter country code (varchar)|
|`country`|Name of the country (varchar)|
|`continent`|Name of the continent the country exists in (varchar)|
---
`categories`
|Column|Description|
|------|-----------|
|`category_code`|Code for the business category (varchar)|
|`category`|Description of the business category (varchar)|



**Goal: Understand the factors that help a business be timeless by answering these questions:**


**Question 1:** What is the oldest business on each continent? 
The results are stored as oldest_business_continent with four columns: continent, country, business, and year_founded in any order.

In [9]:
-- Find the oldest business on each continent

SELECT 
    c.continent,         -- Continent name
    b.year_founded,      -- Year the business was founded
    c.country,           -- Country name
    b.business           -- Business name
FROM countries AS c
INNER JOIN businesses AS b
    ON c.country_code = b.country_code
-- Filter to only include businesses that are the oldest in their continent
WHERE (c.continent, b.year_founded) IN (
    SELECT 
        c2.continent, 
        MIN(b2.year_founded)  -- Find the minimum (oldest) year_founded per continent
    FROM countries AS c2
    INNER JOIN businesses AS b2
        ON c2.country_code = b2.country_code
    GROUP BY c2.continent     -- Group by continent to get oldest per continent
);

,continent,year_founded,country,business
0,Oceania,1809,Australia,Australia Post
1,Europe,803,Austria,St. Peter Stifts Kulinarium
2,Asia,578,Japan,Kongō Gumi
3,North America,1534,Mexico,La Casa de Moneda de México
4,Africa,1772,Mauritius,Mauritius Post
5,South America,1565,Peru,Casa Nacional de Moneda


**Analisys and Insights 1:** The query reveals a mix of businesses that have achieved incredible longevity. The oldest business in the dataset, Kongō Gumi in Asia (founded in 578), is a construction company, reflecting a fundamental human need for building. In Europe, the oldest is St. Peter Stifts Kulinarium (803), a restaurant, highlighting the enduring nature of hospitality.

Interestingly, several of the oldest businesses are related to state-run or essential services, such as Mauritius Post in Africa (1772) and mints in North and South America (1534 and 1565 respectively). This suggests that businesses tied to government functions or essential infrastructure often have the stability to survive for centuries.

**Question 2:** How many countries per continent lack data on the oldest businesses? Does including new_businesses change this?  The number of countries per continent missing business data, including new_businesses.
The results are stored as count_missing with columns continent and countries_without_businesses.

In [10]:
-- Select the continent and count of countries without any businesses or new businesses
SELECT
  c.continent,
  -- Count countries where there are no matching businesses or new businesses
  COUNT(*) FILTER (
    WHERE b.business IS NULL AND b2.business IS NULL
  ) AS countries_without_businesses
FROM countries AS c
-- Left join to businesses to find if a country has any business
LEFT JOIN businesses AS b ON c.country_code = b.country_code
-- Left join to new_businesses to find if a country has any new business
LEFT JOIN new_businesses AS b2 ON c.country_code = b2.country_code
-- Group the results by continent
GROUP BY c.continent;

,continent,countries_without_businesses
0,Africa,3
1,Asia,7
2,South America,3
3,Europe,2
4,North America,5
5,Oceania,10


**Analysis and Insights 2:** Before drawing broad conclusions, it's crucial to assess the completeness of the data. This query shows that the dataset has significant gaps, particularly in Oceania, where 10 countries are missing. Asia and North America also have several missing countries.

**Question 3:** Which business categories are best suited to last many years, and on what continent are they? 
The results are stored as oldest_by_continent_category with the oldest founding year for each continent and category combination. It contains three columns: continent, category, and year_founded, in that order.

In [11]:
-- Find the oldest year_founded for each business category and continent,
-- to identify which business categories have lasted the longest (i.e., are best suited to last over centuries)

SELECT 
    c.continent AS continent,                -- Continent where the business is located
    c2.category AS category,                 -- Business category
    MIN(b.year_founded) AS year_founded      -- Earliest year a business in this category was founded in this      continent
FROM 
    countries AS c
INNER JOIN 
    businesses AS b
    ON c.country_code = b.country_code       -- Join businesses to countries to get continent info
INNER JOIN 
    categories AS c2
    ON b.category_code = c2.category_code    -- Join businesses to categories to get category name
GROUP BY 
    c2.category, c.continent                 -- Group by category and continent
ORDER BY 
    year_founded;                            -- Order by the oldest year_founded (ascending)

,continent,category,year_founded
0,Asia,Construction,578
1,Europe,"Cafés, Restaurants & Bars",803
2,Europe,"Distillers, Vintners, & Breweries",862
3,Europe,Manufacturing & Production,864
4,Asia,"Cafés, Restaurants & Bars",1153
5,Europe,Agriculture,1218
6,Europe,Tourism & Hotels,1230
7,Europe,Mining,1248
8,Europe,Medical,1422
9,Europe,Postal Service,1520


**Final analysis:** This final query provides the clearest picture of which industries are built to last. By ranking categories by their oldest known business, we can see strong patterns:

Foundational Industries: The absolute oldest categories are in sectors like Construction (Asia, 578) and Agriculture (Europe, 1218). These represent primary human needs that are timeless.

Food and Beverage: "Cafés, Restaurants & Bars" and "Distillers, Vintners, & Breweries" appear very early, especially in Europe (803 and 862). This highlights the cultural and social importance of dining and drinking establishments.

Essential Services: Banking, finance, and postal services are also highly resilient categories across multiple continents. This reinforces the idea that industries intertwined with the economy and state infrastructure have a high survival rate.

Overall, the analysis suggests that the businesses most likely to endure for centuries are those that fulfill fundamental societal needs, provide essential services, or are deeply embedded in cultural traditions.

**Source:** datacamp.com